In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [4]:
from datasets import load_dataset

arc_challenge = load_dataset("allenai/ai2_arc", "ARC-Challenge")
arc_easy = load_dataset("allenai/ai2_arc", "ARC-Easy")

In [17]:
arc_challenge['validation'][298]

{'id': 'Mercury_7082145',
 'question': 'How should a line graph be used to display distance and time data for a moving object?',
 'choices': {'text': ['The y-axis should be labeled as time, which is the dependent variable.',
   'The y-axis should be labeled as distance, which is the independent variable.',
   'The x-axis should be labeled as distance, which is the dependent variable.',
   'The x-axis should be labeled as time, which is the independent variable.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D'}

In [5]:
import re
import ast

In [6]:
import amrlib
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
from openai import OpenAI
client = OpenAI(api_key='YOUR_KEY')

# Set up YOUR OpenAI API key
#OPENAI_API_KEY = "YOUR_KEY"
# Set MODEL to "gpt-3.5-turbo"
model = "gpt-3.5-turbo"
# Set temperature to 0.9, it can be from 0~1, 0 is the most conservative, 1 is the most creative
temperature = 0.5

In [8]:
def Answer_Agent_IO(question='', candidates='', num_seq=1):
    message = \
    f"""Given a question, give out the answer from the candidates, without any explanations. \
    Here is an example as guideline:
    "Question: What is the capital of France? \
    Candidates: A) London; B) Paris; C) Berlin; D) Madrid; \
    Answer: ###B###" \
    Make sure to include your answer in the format: ###your choice###. \
    Question: {question} \
    Candidates: {candidates} \
    Answer: \
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages_in, temperature=temperature, n=num_seq)

    # response = response_message.choices[1].message.content

    # return response

    responses = []
    n = 0
    while n < num_seq:
        responses.append(response_message.choices[n].message.content)
        n += 1
    
    return responses

In [9]:
def Answer_Agent_CoT_0Shot(question='', candidates='', num_seq=1):
    message = \
    f"""Given a question, give out the answer from the candidates, without any explanations. \
    Make sure to include your answer in the format: ###your choice###. \
    Question: {question} \
    Candidates: {candidates} \
    Let's think step by step. \
    Answer: \
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that answer scientific questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages_in, temperature=temperature, n=num_seq)

    # response = response_message.choices[1].message.content

    # return response

    responses = []
    n = 0
    while n < num_seq:
        responses.append(response_message.choices[n].message.content)
        n += 1
    
    return responses

In [10]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [11]:
ord('A')

65

In [16]:
all_answer_lm3_text_io = []
ori_answers_io = []
all_counts = -1
for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_IO(ori_question, choices_text, 1)[0]
    ori_answers_io.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_io.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_io.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_io.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_io.append('')
                continue

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:48<00:00,  2.07it/s]


In [17]:
all_answer_lm3_text_cot_0s = []
all_counts = -1

for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_CoT_0Shot(ori_question, choices_text, 1)[0]
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot_0s.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot_0s.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot_0s.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot_0s.append('')
                continue

100%|██████████| 100/100 [00:37<00:00,  2.65it/s]


In [10]:
import sys
import re
import string
from collections import Counter
import pickle

In [11]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [20]:
with open('answers_gpt3_io.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_io))

In [21]:
with open('answers_gpt3_io_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_answers_io))

In [22]:
with open('answers_gpt3_cot_0Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot_0s))

In [40]:
with open('answers_lm3_cot_2Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot))

In [38]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['answerKey'] for s in list(arc_challenge['validation'])[0:100]))

In [12]:
with open('answers_golden.txt', 'r') as file:
    golden = file.readlines()

In [24]:
with open('answers_gpt3_io.txt', 'r') as file:
    output_io = file.readlines()

In [21]:
with open('answers_gpt3_cot_0Shot.txt', 'r') as file:
    output_cot = file.readlines()

In [18]:
def evaluation(output, golden):
    correct = 0
    length = len(golden)

    for (o, g) in zip(output, golden):
        o = o[0]
        g = g[0]
        if o == g:
            correct += 1
    
    return correct, correct/length


In [38]:
evaluation(output_io, golden)

(68, 0.68)

In [39]:
evaluation(output_cot, golden)

(82, 0.82)

In [67]:
def Answer_Agent_debate(ori_question='', candidates='', rounds=1, num_agents=3):
    message_user_prior = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###your answer###. \
    Question: {ori_question} \
    Let's think step by step. \
    Answer: \
    """

    message = \
    f"""Given some potential answers given by other agents: \
    if you think your previous answer is wrong, and one of the potential answers is correct, \
    provide the correct answer and your reasoning for it; \
    if you think your previous answer is correct, explain why it is correct. \
    Make sure to include answer choice (A, B, C, or D) in your final answer. \
    Make sure to include your final answer in ###, as shown here: ###Answer: your choice of answer###. \
    """
    #    Question: {ori_question} \

    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer scientific questions accurately."}]
    
    all_responses = {}
    for i in range(num_agents):
        all_responses[i] = {'user': [], 'assistant': []}
    response_pool = set()
    
    response_list = Answer_Agent_CoT_0Shot(ori_question, candidates, num_agents)
    for j in range(len(response_list)):
        all_responses[j]['user'].append(message_user_prior)
        try:
            response_format = re.search('###(.*)###', response_list[j]).group(1)
            all_responses[j]['assistant'].append(response_format)
            response_pool.add(response_format)
        except:
            response_format = response_list[j]
            response_pool.add(response_format)
    
    for r in range(rounds):
        if len(response_pool) == 1:
            break
        for i in range(num_agents):
            new_messages = messages
            new_message = message

            new_messages.append({"role": "user", "content": all_responses[i]['user'][0]})
            new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][0]})
            
            if r != 0:
                new_messages.append({"role": "user", "content": all_responses[i]['user'][r]})
                new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][r]})

            potential_answers = list(response_pool)
            new_message += f'Potential answers: '
            num_ans = 1
            for pa in potential_answers:
                new_message += f'{num_ans}:{pa};'
                num_ans += 1
            new_messages.append({"role": "user", "content": new_message})
            
            # other_responses = response_list[:i] + response_list[i+1:]
            # message += f'Reasoning from other agents: '
            # for k in range(len(other_responses)):
            #     message += f'Agent {k+1}: {other_responses[k]}'
            # message += 'Thought:'
            # new_messages.append({"role": "user", "content": message})
            
            response_message = client.chat.completions.create(model=model, messages=new_messages, temperature=temperature)
            response = response_message.choices[0].message.content
            all_responses[i]['user'].append(new_message)
            all_responses[i]['assistant'].append(response)

    #put all answers in a pool, and then do majority vote
    all_responses_pool = []
    for i in range(num_agents):
        answer_i = all_responses[i]['assistant'][-1]
        try:
            response_format = re.search('###(.*)###', answer_i).group(1)
            all_responses_pool.append(response_format)
        except:
            all_responses_pool.append(answer_i)

    return all_responses, max(set(all_responses_pool), key=all_responses_pool.count)

In [68]:
all_answer_gpt3_text_cot_0s_3ensemble = []
oris = []
all_counts = -1

for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response, final_response = Answer_Agent_debate(ori_question, choices_text, 2, 3)
    all_answer_gpt3_text_cot_0s_3ensemble.append(final_response)
    oris.append(response)

100%|██████████| 100/100 [00:45<00:00,  2.22it/s]


In [66]:
arc_challenge['validation'][7]

{'id': 'MDSA_2007_4_52',
 'question': 'Plants and animals need food for growth. What happens to most of the food that plants produce?',
 'choices': {'text': ['Food is released as gas.',
   'Food is converted to water.',
   'Food is stored for future use.',
   'Food is used to absorb sunlight.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'C'}

In [71]:
oris

[{0: {'user': ["Given a question, give out your chain of thought process and the final answer.     Make sure to include your answer in the format: ###your answer###.     Question: Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?     Let's think step by step.     Answer:     "],
   'assistant': ['D) Record the details of the investigation.']},
  1: {'user': ["Given a question, give out your chain of thought process and the final answer.     Make sure to include your answer in the format: ###your answer###.     Question: Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?     Let's think step by step.     Answer:     "],
   'assistant': ['D) Record the details of the investigation.']},
  2: {'user': ["Given a question, give out your chain of thought process and the final a

In [73]:
c, r, l = new_evaluation(all_answer_gpt3_text_cot_0s_3ensemble, golden)

In [74]:
c, r

(84, 0.84)

In [72]:
def new_evaluation(output, golden):
    correct = 0
    length = len(golden)
    new_os = []

    for (o, g) in zip(output, golden):
        if o[-1] == ')':
            o = o[-2]
        else:
            if ")" in o:
                o = o.split(')')[0].strip()[0]
            #o = o.split(':')[1].strip()[0]
        g = g[0]
        if o == g:
            correct += 1
        new_os.append(o)
    
    return correct, correct/length, new_os


In [61]:
c, r, l = new_evaluation(all_answer_gpt3_text_cot_0s_3ensemble, golden)

In [63]:
all_answer_gpt3_text_cot_0s_3ensemble

['Answer: D) Record the details of the investigation',
 'Answer: C) drought',
 'Answer: D) Earth rotates on its axis',
 'Answer: A) converting sunlight into electricity',
 'Answer: B',
 'Answer: C) hunting strategies',
 'Answer: C) The particles move more rapidly',
 'Answer: D) It is used for energy by the plant',
 'Answer: C) amount of garlic',
 'Answer: C) predicting future events',
 'Answer: C) The temperature of the liquid decreased, or the liquid became a gas',
 'Answer: D) the supplements',
 'Answer: A) individual subjective feelings of fatigue',
 'Answer: B) a decrease in biodiversity',
 'Answer: A) tasting chemical samples',
 'Answer: C) The hot water dissolved the most sugar.',
 'Answer: B) the population of producers in the area',
 'Answer: B) The liver',
 'Answer: B) their ability to fly',
 'Answer: B) an increase in the availability of water',
 'Answer: C) natural selection',
 'Answer: B) Broadleaf species would replace pine species',
 'Answer: D) The planet lacks an atmosp

In [51]:
new_ans = []
for aa in oris:
    try:
        ans = re.search('###(.*)###', aa[0]['assistant'][-1]).group(1)
        if ans == 'answer':
            raise Exception('wrong format')
        new_ans.append(ans)
    except:
        try:
            ans = re.search('###(.*)###', aa[1]['assistant'][-1]).group(1)
            if ans == 'answer':
                raise Exception('wrong format')
            new_ans.append(ans)
        except:
            try:
                ans = re.search('###(.*)###', aa[2]['assistant'][-1]).group(1)
                new_ans.append(ans)
            except:
                new_ans.append('A')

In [58]:
new_evaluation(new_ans, golden)

(64, 0.64)